<a href="https://colab.research.google.com/github/JakeTurner616/Adonalsium/blob/LLM/trainandgenerate_1_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [1]:
! pip install  torch pandas
! pip install accelerate
! pip install  transformers
! pip install -U deepspeed
! pip install -U mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.1 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.13.2-py3-none-any.whl size=1360167 sha256=fc229eb6a2becba0e004d7af48d2d3af94d894cc12883021f1ccafae8b47bc86
  Stored in directory: /root/.cache/pip/wheels/a8/78/a8/62089b9f05586da0176ff0c959bdb756c57f9d44a4fa63d2a6
Successfully built deepspeed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.

setup deepspeed config

In [2]:
deepspeed_config = {
  "fp16": {
    "enabled": True
  },
  "zero_optimization": {
    "stage": 2,
    "offload_optimizer": {
      "device": "cpu"
    }
  },
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "gradient_accumulation_steps": 1
}




# Write the configuration to a file
with open('ds_config.json', 'w') as fp:
    import json
    json.dump(deepspeed_config, fp)

Download the book

In [3]:
import requests

# URLs for the data
book_url = 'https://gist.githubusercontent.com/JakeTurner616/7c5bcb37d861ba6fdb66d12a9bb9a084/raw/63d14a3aeed74a23b7aa1dba263b42d7c2b8676e/book.txt'  # Change to your book's URL

# Download the book
book_response = requests.get(book_url)
book_text = book_response.text

# Save the book text to a file (optional, you can also directly use `book_text` for training)
with open('reverted_book.txt', 'w') as f:
    f.write(book_text)

print("Downloaded and saved the book text.")

Downloaded and saved the book text.


Train the model

In [4]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

# Define the model and tokenizer
model_name = 'EleutherAI/gpt-neo-1.3B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Ensure compatibility

# Prepare the dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="reverted_book.txt",
    block_size=128  # Adjust based on memory constraints
)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)


training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,  # Updated based on GPU capacity
    fp16=True,
    deepspeed="ds_config.json",  # Ensure this points to your updated DeepSpeed config
    save_steps=500,
    save_total_limit=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Time to load cpu_adam op: 39.224708795547485 seconds


/usr/local/lib/python3.10/dist-packages/deepspeed/runtime/zero/stage_1_and_2.py:1990: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  overflow_gpu = get_accelerator().ByteTensor([overflow])


Step,Training Loss
500,2.592800
1000,2.382900


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1879: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1879: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
#from google.colab import files
#files.download('/content/results/checkpoint-1000/zero_to_fp32.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generate text

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def generate_text(prompt, model_name='JakeTurner616/Adonalsium-gpt-neo-1.3B'):  # Replace with your actual model name on Hugging Face
    # Load the model and tokenizer from Hugging Face Model Hub
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Create a text generation pipeline using the loaded model and tokenizer
    text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

    # Generate text based on the provided prompt
    generated_texts = text_generator(
        prompt,
        max_length=150,
        temperature=0.7,
        top_p=0.85,
        repetition_penalty=1.3,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Print the generated texts
    for generated in generated_texts:
        print(generated['generated_text'])

if __name__ == "__main__":
    prompt = "She fell to the ground dead"
    generate_text(prompt)

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


She fell to the ground dead, and now you’re here. You claim this is all a misunderstanding, but I see only one man who could have been behind it. The assassin in White Sand. He was known to many of us, even among my kind.”
 
 “He died when we caught him trying to steal from me. What are you saying? That he was working with someone else? A Mistborn?“
 She shook her head, looking up at Pattern on the floor before them. ‘No. It‘s impossible. No mortal would be capable of that sort of thing. They‚ you know … they don„‛t work like ours do. We
